<a href="https://colab.research.google.com/github/nkalra0123/kokoro-tts/blob/main/1_instruction_tuning/notebooks/sft_finetuning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Install the requirements in Google Colab
!pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-DeepSeek-R1"
finetune_tags = ["smol-course", "module_1"]

# Generate with the base model

Here we will try out the base model which does not have a chat template.

In [6]:
# Let's test the base model before training
prompt = "Write a essay about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=500)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
system
You are a helpful AI assistant named SmolLM, trained by Hugging Face
user
Write a essay about programming
assistant
Programming is a fundamental skill that has revolutionized the way we interact with technology. From the simple task of writing a program to the complex task of creating a game, programming has become an essential part of our daily lives.

In the early days of programming, it was not possible to write a program without a programming language. The first programming language was called "Lisp," which was developed by William F. Brooks in the 1960s. Lisp was a simple language that was easy to learn and used by many programmers.

However, as the years went by, the need for a programming language became more pressing. The need for a programming language became more urgent because of the increasing complexity of the world around us. The internet, social media, and the world of software development became more and more complex.

The first programming langu

## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [7]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="mlabonne/dolphin-r1-deepseek")

README.md:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

train-00000-of-00008.parquet:   0%|          | 0.00/208M [00:00<?, ?B/s]

train-00001-of-00008.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

train-00002-of-00008.parquet:   0%|          | 0.00/209M [00:00<?, ?B/s]

train-00003-of-00008.parquet:   0%|          | 0.00/207M [00:00<?, ?B/s]

train-00004-of-00008.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

train-00005-of-00008.parquet:   0%|          | 0.00/209M [00:00<?, ?B/s]

train-00006-of-00008.parquet:   0%|          | 0.00/209M [00:00<?, ?B/s]

train-00007-of-00008.parquet:   0%|          | 0.00/211M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/300018 [00:00<?, ? examples/s]

In [8]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)
ds

DatasetDict({
    train: Dataset({
        features: ['messages', 'model'],
        num_rows: 300018
    })
})

In [9]:
import random

def downsample(data, fraction):
    num_samples = int(len(data) * fraction)
    indices = list(range(len(data)))
    selected_indices = sorted(random.sample(indices, num_samples))
    return data.select(selected_indices)

In [10]:
ds['train'] = downsample(ds['train'], 0.5)


In [11]:
ds

DatasetDict({
    train: Dataset({
        features: ['messages', 'model'],
        num_rows: 150009
    })
})

In [14]:
ds['train'][0]['messages']

[{'content': 'Provide a concise, objective summary of the input text in up to three sentences, focusing on key actions and intentions without using second or third person pronouns.',
  'role': 'system'},
 {'content': 'Dear Pierre,\n\nI hope this email finds you well. I\'m excited to inform you that I\'ve been invited to give a lecture at the "Baroque Art in the Age of the Counter-Reformation" conference in Paris next month. I can\'t wait to visit your beautiful city and reconnect with you after all these years!\n\nI was wondering if you could recommend some places I should visit during my stay, especially any museums or galleries that have significant Baroque art collections. Also, do you have any suggestions for restaurants or cafes where we could catch up?\n\nLooking forward to your thoughts and seeing you soon!\n\nBest regards,\nMaria',
  'role': 'user'},
 {'content': '<|begin_of_thought|>\n\nOkay, I need to summarize this email from Maria to Pierre. The user wants a concise, object

In [15]:
ds = ds['train'].train_test_split(test_size=0.1)


In [16]:
ds

DatasetDict({
    train: Dataset({
        features: ['messages', 'model'],
        num_rows: 135008
    })
    test: Dataset({
        features: ['messages', 'model'],
        num_rows: 15001
    })
})

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [ ]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-17-05f22f6eec0e>:18: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/135008 [00:00<?, ? examples/s]

## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [ ]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

In [ ]:
trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [ ]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.